In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime

from tqdm import tqdm
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
from sklearn.metrics import mean_squared_error

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']


### Data Load

In [2]:
# df_train = utils.read_df_pkl('../input/train0*')
# df_test = utils.read_df_pkl('../input/test0*')
df_hist = utils.read_df_pkl('../input/historical*0*')
df_new = utils.read_df_pkl('../input/new_mer*0*')

df_hist['purchase_amount'] -= df_hist['purchase_amount'].min() + 1.0e-4
df_new['purchase_amount'] -= df_new['purchase_amount'].min() + 1.0e-4
df_hist['installments'] =  df_hist['installments'].map(lambda x: np.nan if x<0 or x>100 else x+1)
df_hist['installments'] =  df_hist['installments'].map(lambda x: np.nan if x<0 or x>100 else x+1)

100%|██████████| 3/3 [00:00<00:00,  5.28it/s]


In [3]:
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

In [4]:
for df in [df_hist,df_new]:
# for df in [df_hist]:
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['year'] = df['purchase_date'].dt.year
    df['month'] = df['purchase_date'].dt.month
    df['dayofweek'] = df['purchase_date'].dt.dayofweek
    df['hour'] = df['purchase_date'].dt.hour
    df['authorized_flag'] = df['authorized_flag'].map({'Y':1, 'N':0})
    df['category_1'] = df['category_1'].map({'Y':1, 'N':0}) 
    #https://www.kaggle.com/c/elo-merchant-category-recommendation/discussion/73244
    df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']

In [5]:
auth1_hist = df_hist[df_hist.authorized_flag==1]
auth0_hist = df_hist[df_hist.authorized_flag==0]

In [6]:
new_aggs = {}
new_aggs['purchase_date'] = ['max','min']

new_columns = get_new_columns('hist', new_aggs)
tmp = df_hist[['card_id', 'purchase_date']].groupby('card_id').agg(new_aggs)
tmp.columns = new_columns

tmp['hist_term'] = (tmp['hist_purchase_date_max'] - tmp['hist_purchase_date_min']).dt.days

### 117 auth1_hist feature

In [7]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')

df = auth1_hist
fname = 'auth1_hist'
aggs = {}

aggs['month_lag'] = ['mean', 'var']
aggs['month'] = ['nunique']
# aggs['dayofweek'] = ['nunique']
aggs['purchase_amount'] = ['sum','max', 'min']
aggs['installments'] = ['max', 'min','mean']
aggs['purchase_date'] = ['max','min']
aggs['month_diff'] = ['mean']
aggs['category_1'] = ['sum', 'mean']
aggs['card_id'] = ['size']

# for col in ['category_2','category_3']:
#     df[col+'_mean'] = df.groupby([col])['purchase_amount'].transform('mean')
#     aggs[col+'_mean'] = ['mean']    

new_columns = get_new_columns(fname, aggs)
df_agg = df.groupby('card_id').agg(aggs)
df_agg.columns = new_columns
df_agg.reset_index(drop=False,inplace=True)

# df_agg['auth1_hist_term'] = (df_agg['auth1_hist_purchase_date_max'] - df_agg['auth1_hist_purchase_date_min']).dt.days
# df_agg = df_agg.merge(tmp, how='inner', on=key)
# df_agg[f'{fname}_term_ratio'] = df_agg['auth1_hist_term'] / df_agg['hist_term']

# df_agg['auth1_hist_term_freq'] = df_agg['auth1_hist_card_id_size']/df_agg['auth1_hist_term']
df_train = df_train.merge(df_agg,on='card_id',how='left')
df_test = df_test.merge(df_agg,on='card_id',how='left')
del df_agg
gc.collect()
df_train.head()

100%|██████████| 3/3 [00:00<00:00, 174.19it/s]


,first_active_month,card_id,feature_1,feature_2,feature_3,target,auth1_hist_month_lag_mean,auth1_hist_month_lag_var,auth1_hist_month_nunique,auth1_hist_purchase_amount_sum,auth1_hist_purchase_amount_max,auth1_hist_purchase_amount_min,auth1_hist_installments_max,auth1_hist_installments_min,auth1_hist_installments_mean,auth1_hist_purchase_date_max,auth1_hist_purchase_date_min,auth1_hist_month_diff_mean,auth1_hist_category_1_sum,auth1_hist_category_1_mean,auth1_hist_card_id_size
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,-3.882591,5.900793,9,27.064503,3.005202,0.007413,2.0,2.0,2.000000,2018-02-25 09:31:15,2017-06-27 14:18:08,11.542510,0,0.000000,247
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,-5.050147,14.722330,12,44.284384,5.377107,0.004408,12.0,3.0,3.492582,2018-01-31 22:31:09,2017-01-06 16:29:42,12.498525,29,0.085546,339
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,-8.487805,15.156098,10,2.790695,0.600960,0.016670,2.0,2.0,2.000000,2018-02-27 19:08:25,2017-01-11 08:21:22,11.341463,0,0.000000,41
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495,-2.831169,3.247437,6,8.012834,2.192403,0.005911,5.0,3.0,3.146667,2018-02-28 11:44:40,2017-09-26 16:22:21,11.428571,12,0.155844,77
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749,-1.320312,1.054072,4,26.550929,7.739425,0.000651,14.0,3.0,3.125000,2018-02-28 20:40:41,2017-11-12 00:00:00,11.484375,12,0.093750,128


In [13]:
# feat_no = '117_ah1_'
prefix = '152_ah1_'
prefix = '352_ah1_'
feat_no = '452_ah1_'
ignore_features = ['first_active_month', 'card_id', target]

df_feat = pd.concat([df_train, df_test], axis=0)

for col in df_feat.columns:
    if col in ignore_features: continue
    if (col.count('feature_')):continue
#     if (col.count('auth0_')):continue
#     if not(col.count('install')):continue
    feature = df_feat[col].values.astype('float32')
    feature = df_feat[col].fillna(-1).values.astype('float32')
    col = col.replace('.', '_')
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{prefix}_{col}@', obj=feature)

### 118 auth0_hist feature

In [14]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')

df = auth0_hist
fname = 'auth0_hist'
aggs = {}

aggs['month_lag'] = ['mean', 'var']
aggs['purchase_amount'] = ['sum','max', 'min']
aggs['installments'] = ['max', 'min','mean']
aggs['purchase_date'] = ['max','min']
aggs['month_diff'] = ['mean']
aggs['category_1'] = ['sum', 'mean']
aggs['card_id'] = ['size']

# for col in ['category_2','category_3']:
#     df[col+'_mean'] = df.groupby([col])['purchase_amount'].transform('mean')
#     aggs[col+'_mean'] = ['mean']    

new_columns = get_new_columns(fname, aggs)
df_agg = df.groupby('card_id').agg(aggs)
df_agg.columns = new_columns
df_agg.reset_index(drop=False,inplace=True)

# df_agg[f'{fname}_term'] = (df_agg[f'{fname}_purchase_date_max'] - df_agg[f'{fname}_purchase_date_min']).dt.days
# df_agg = df_agg.merge(tmp, how='inner', on=key)
# df_agg[f'{fname}_term_ratio'] = df_agg['auth0_hist_term'] / df_agg['hist_term']

# df_agg[f'{fname}_term_freq'] = df_agg[f'{fname}_card_id_size']/df_agg[f'{fname}_term']
df_train = df_train.merge(df_agg,on='card_id',how='left')
df_test = df_test.merge(df_agg,on='card_id',how='left')
del df_agg
gc.collect()
df_train.head()

100%|██████████| 3/3 [00:00<00:00, 177.98it/s]


,first_active_month,card_id,feature_1,feature_2,feature_3,target,auth0_hist_month_lag_mean,auth0_hist_month_lag_var,auth0_hist_purchase_amount_sum,auth0_hist_purchase_amount_max,auth0_hist_purchase_amount_min,auth0_hist_installments_max,auth0_hist_installments_min,auth0_hist_installments_mean,auth0_hist_purchase_date_max,auth0_hist_purchase_date_min,auth0_hist_month_diff_mean,auth0_hist_category_1_sum,auth0_hist_category_1_mean,auth0_hist_card_id_size
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,-4.461538,2.769231,1.136778,0.314886,0.008916,3.0,2.0,2.307692,2017-12-27 14:33:33,2017-07-15 14:58:10,11.769231,0.0,0.000000,13.0
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,-4.454545,7.272727,7.091999,2.689646,0.005911,12.0,3.0,5.818182,2017-12-28 13:21:18,2017-03-03 21:27:17,12.454545,2.0,0.181818,11.0
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,-11.000000,2.000000,0.154648,0.109293,0.045355,2.0,2.0,2.000000,2017-04-21 19:00:14,2017-02-23 19:32:09,11.000000,0.0,0.000000,2.0
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749,-0.400000,0.300000,24.086848,7.939849,0.233863,14.0,3.0,9.600000,2018-02-27 19:21:23,2018-01-20 21:49:13,11.000000,3.0,0.600000,5.0


In [16]:
for col in df_train.columns:
    print(df_train[col].dropna().shape)
    print(col, df_train[col].max(), df_train[col].min())

(201917,)
first_active_month 2018-02 2011-11
(201917,)
card_id C_ID_fffffd5772 C_ID_00007093c1
(201917,)
feature_1 5 1
(201917,)
feature_2 3 1
(201917,)
feature_3 1 0
(201917,)
target 17.9650684 -33.21928095
(170958,)
auth0_hist_month_lag_mean 0.0 -13.0
(149929,)
auth0_hist_month_lag_var 84.5 0.0
(170958,)
auth0_hist_purchase_amount_sum 6010604.746807771 -0.0034841648101802125
(170958,)
auth0_hist_purchase_amount_max 6010604.746807771 -8.497962951659055e-05
(170958,)
auth0_hist_purchase_amount_min 6010604.746807771 -9.999999999998899e-05
(170402,)
auth0_hist_installments_max 14.0 2.0
(170402,)
auth0_hist_installments_min 14.0 2.0
(170402,)
auth0_hist_installments_mean 14.0 2.0
(170958,)
auth0_hist_purchase_date_max 2018-02-28 23:59:43 2017-01-01 00:08:31
(170958,)
auth0_hist_purchase_date_min 2018-02-28 23:35:45 2017-01-01 00:02:14
(170958,)
auth0_hist_month_diff_mean 24.0 11.0
(170958,)
auth0_hist_category_1_sum 1510.0 0.0
(170958,)
auth0_hist_category_1_mean 1.0 0.0
(170958,)
auth0_h

In [19]:
prefix = '152_ah0_'
prefix = '352_ah0_'
prefix = '452_ah0_'
ignore_features = ['first_active_month', 'card_id', target]

df_feat = pd.concat([df_train, df_test], axis=0)

for col in df_feat.columns:
    if col in ignore_features: continue
    if (col.count('feature_')):continue
    if (col.count('purchase_date')):continue
#     if not(col.count('install')):continue
#     feature = df_feat[col].values.astype('float32')
    if col.count('month_lag'):
        val_min = df_feat[col].min()
        feature = df_feat[col].fillna(val_min-1).values.astype('float32')
    elif col.count('month_diff'):
        val_max = df_feat[col].max()
        feature = df_feat[col].fillna(val_max+1).values.astype('float32')
    else:
        feature = df_feat[col].fillna(-1).values.astype('float32')
    col = col.replace('.', '_')
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{prefix}_{col}@', obj=feature)

### 119 auth1 auth0 date max, min diff

In [64]:
feat_no = '119_ath_'
df_train['auth1_0_hist_purchase_date_max_diff'] = (df_train['auth1_hist_purchase_date_max'] -  df_train['auth0_hist_purchase_date_max']).map(lambda x: x.days)
df_train['auth1_0_hist_purchase_date_min_diff'] = (df_train['auth1_hist_purchase_date_min'] -  df_train['auth0_hist_purchase_date_min']).map(lambda x: x.days)
df_test['auth1_0_hist_purchase_date_max_diff'] = (df_test['auth1_hist_purchase_date_max'] -  df_test['auth0_hist_purchase_date_max']).map(lambda x: x.days)
df_test['auth1_0_hist_purchase_date_min_diff'] = (df_test['auth1_hist_purchase_date_min'] -  df_test['auth0_hist_purchase_date_min']).map(lambda x: x.days)

ignore_features = ['first_active_month', 'card_id', target]

for col in df_train.columns:
    if col in ignore_features: continue
    if not(col.count('auth1_0_hist')):continue
#     if not(col.count('card_id_total')):continue
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{feat_no}train_{col}@', obj=df_train[col].values)
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{feat_no}test_{col}@', obj=df_test[col].values)

In [31]:
new_aggs = {}
new_aggs['authorized_flag'] = ['sum', 'mean']

SystemExit: 

### 120 初期3ヶ月のauthorized_meanと直近3ヶ月のauthorized_mean, そのdiff

In [5]:
df_hist['yyyymm'] = df_hist['purchase_date'].map(lambda x: str(x)[:7])
df_hist.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,year,month,dayofweek,hour,month_diff,yyyymm
0,1,C_ID_4e6213e9bc,88,0,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37,2017,6,6,15,10,2017-06
1,1,C_ID_4e6213e9bc,88,0,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16,2017,7,5,12,10,2017-07
2,1,C_ID_4e6213e9bc,88,0,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37,2017,8,2,22,10,2017-08
3,1,C_ID_4e6213e9bc,88,0,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34,2017,9,5,10,11,2017-09
4,1,C_ID_4e6213e9bc,88,0,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37,2017,3,4,1,11,2017-03


In [6]:
# 初期3ヶ月
df_auth = df_hist[[key, 'yyyymm', 'authorized_flag']]
df_auth.sort_values(by=[key, 'yyyymm'], inplace=True)
df_auth_row = utils.row_number(df=df_auth[[key, 'yyyymm']].drop_duplicates(), level=[key])
df_auth_row = df_auth_row.merge(df_hist[[key, 'yyyymm', 'authorized_flag']], how='inner', on=[key, 'yyyymm'])

df_first3 = df_auth_row[df_auth_row['row_no']<=3]

df_first3.head(10)

,card_id,yyyymm,row_no,authorized_flag
0,C_ID_00007093c1,2017-02,1,1
1,C_ID_00007093c1,2017-02,1,1
2,C_ID_00007093c1,2017-02,1,1
3,C_ID_00007093c1,2017-02,1,1
4,C_ID_00007093c1,2017-03,2,1
5,C_ID_00007093c1,2017-03,2,1
6,C_ID_00007093c1,2017-03,2,1
7,C_ID_00007093c1,2017-03,2,1
8,C_ID_00007093c1,2017-03,2,0
9,C_ID_00007093c1,2017-03,2,0


In [7]:
# 後期3ヶ月
# df_auth = df_hist[[key, 'yyyymm', 'authorized_flag']]
df_auth.sort_values(by=[key, 'yyyymm'], ascending=False, inplace=True)
df_auth_row = utils.row_number(df=df_auth[[key, 'yyyymm']].drop_duplicates(), level=[key])
df_auth_row = df_auth_row.merge(df_hist[[key, 'yyyymm', 'authorized_flag']], how='inner', on=[key, 'yyyymm'])

df_last3 = df_auth_row[df_auth_row['row_no']<=3]

df_last3.head(10)

,card_id,yyyymm,row_no,authorized_flag
0,C_ID_fffffd5772,2018-02,1,1
1,C_ID_fffffd5772,2018-02,1,1
2,C_ID_fffffd5772,2018-02,1,1
3,C_ID_fffffd5772,2018-02,1,1
4,C_ID_fffffd5772,2018-02,1,0
5,C_ID_fffffd5772,2018-02,1,0
6,C_ID_fffffd5772,2018-02,1,0
7,C_ID_fffffd5772,2018-02,1,1
8,C_ID_fffffd5772,2018-02,1,0
9,C_ID_fffffd5772,2018-02,1,1


In [12]:
# df_first3_auth = df_first3.groupby(key)['authorized_flag'].mean()
# df_first3_auth.name = 'first_3month_authorized_flag_mean'

# df_last3_auth = df_last3.groupby(key)['authorized_flag'].mean()
# df_last3_auth.name = 'last_3month_authorized_flag_mean'

df_train = df_train.merge(df_first3_auth.reset_index(), how='left', on=key)
df_train = df_train.merge(df_last3_auth.reset_index(), how='left', on=key)
df_test = df_test.merge(df_first3_auth.reset_index(), how='left', on=key)
df_test = df_test.merge(df_last3_auth.reset_index(), how='left', on=key)

df_train['diff_last_first_authorized_flag_mean'] = df_train['last_3month_authorized_flag_mean'] -  df_train['first_3month_authorized_flag_mean']
df_test['diff_last_first_authorized_flag_mean'] = df_test['last_3month_authorized_flag_mean'] -  df_test['first_3month_authorized_flag_mean']

df_train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,first_3month_authorized_flag_mean,last_3month_authorized_flag_mean,diff_last_first_authorized_flag_mean
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,0.947917,0.980198,0.032281
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,0.982143,0.982456,0.000313
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,0.941176,1.000000,0.058824
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495,1.000000,1.000000,0.000000
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749,0.979592,0.955357,-0.024235


In [14]:
feat_no = '120_ath_'

ignore_features = ['first_active_month', 'card_id', target]

for col in df_train.columns:
    if col in ignore_features: continue
    if not(col.count('flag_mean')):continue
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{feat_no}train_{col}@', obj=df_train[col].values)
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{feat_no}test_{col}@', obj=df_test[col].values)

### 152 new amount

In [20]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')

df = df_new
fname = 'new'
aggs = {}

aggs['month_lag'] = ['mean', 'var']
aggs['month'] = ['nunique']
# aggs['dayofweek'] = ['nunique']
aggs['purchase_amount'] = ['sum','max', 'min']
aggs['installments'] = ['max', 'min','mean']
# aggs['purchase_date'] = ['max','min']
aggs['month_diff'] = ['mean']
aggs['category_1'] = ['sum', 'mean']
aggs['card_id'] = ['size']

# for col in ['category_2','category_3']:
#     df[col+'_mean'] = df.groupby([col])['purchase_amount'].transform('mean')
#     aggs[col+'_mean'] = ['mean']    

new_columns = get_new_columns(fname, aggs)
df_agg = df.groupby('card_id').agg(aggs)
df_agg.columns = new_columns
df_agg.reset_index(drop=False,inplace=True)

# df_agg['auth1_hist_term'] = (df_agg['auth1_hist_purchase_date_max'] - df_agg['auth1_hist_purchase_date_min']).dt.days
# df_agg = df_agg.merge(tmp, how='inner', on=key)
# df_agg[f'{fname}_term_ratio'] = df_agg['auth1_hist_term'] / df_agg['hist_term']

# df_agg['auth1_hist_term_freq'] = df_agg['auth1_hist_card_id_size']/df_agg['auth1_hist_term']
df_train = df_train.merge(df_agg,on='card_id',how='left')
df_test = df_test.merge(df_agg,on='card_id',how='left')
del df_agg
gc.collect()
df_train.head()

100%|██████████| 3/3 [00:00<00:00, 175.24it/s]


,first_active_month,card_id,feature_1,feature_2,feature_3,target,new_month_lag_mean,new_month_lag_var,new_month_nunique,new_purchase_amount_sum,new_purchase_amount_max,new_purchase_amount_min,new_installments_max,new_installments_min,new_installments_mean,new_month_diff_mean,new_category_1_sum,new_category_1_mean,new_card_id_size
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,1.478261,0.260870,2.0,3.939453,0.450928,0.022461,0.0,0.0,0.000000,11.521739,0.0,0.000000,23.0
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,1.500000,0.300000,2.0,0.126953,0.045410,0.007812,1.0,1.0,1.000000,12.500000,0.0,0.000000,6.0
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,2.000000,NaN,1.0,0.046875,0.046875,0.046875,0.0,0.0,0.000000,11.000000,0.0,0.000000,1.0
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495,1.714286,0.238095,2.0,0.574707,0.180176,0.012695,1.0,-1.0,0.714286,11.571429,1.0,0.142857,7.0
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749,1.555556,0.253968,2.0,6.968750,1.198242,0.007812,2.0,-1.0,0.972222,11.583333,2.0,0.055556,36.0


In [25]:
prefix = '152'
prefix = '352'
prefix = '452'
 
save_elo_feature(prefix)